In [10]:
%pip install geopandas pandas pyshp


Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import geopandas as gpd

In [12]:
df = pd.read_csv('CO2 emission by countries.csv')
df.head()

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km�
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km�
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km�
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km�
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km�


In [13]:
df = df.loc[:, ['Country', 'Year', 'CO2 emission (Tons)']]
df.head()

,Country,Year,CO2 emission (Tons)
0,Afghanistan,1750,0.0
1,Afghanistan,1751,0.0
2,Afghanistan,1752,0.0
3,Afghanistan,1753,0.0
4,Afghanistan,1754,0.0


In [14]:
total_world_emissions = df.groupby('Year')['CO2 emission (Tons)'].sum().reset_index()
total_world_emissions = total_world_emissions.rename(columns={'CO2 emission (Tons)': 'Total World Emission (Tons)'})
total_world_emissions.head()

,Year,Total World Emission (Tons)
0,1750,9350528.0
1,1751,18701056.0
2,1752,28055248.0
3,1753,37409440.0
4,1754,46767296.0


In [15]:
df = df.merge(total_world_emissions, on='Year')
# # Rename the 'CO2 emission (Tons)' column from df2
# df = df.rename(columns={'Total World Emission (Tons)_y': 'Total World Emission (Tons)'})
# # Drop the duplicate 'CO2 emission (Tons)' column
# df = df.drop(columns=['Total World Emission (Tons)_x'])
# Display the final DataFrame with formatting
pd.options.display.float_format = '{:.1f}'.format
df.head()

,Country,Year,CO2 emission (Tons),Total World Emission (Tons)
0,Afghanistan,1750,0.0,9350528.0
1,Albania,1750,0.0,9350528.0
2,Algeria,1750,0.0,9350528.0
3,Andorra,1750,0.0,9350528.0
4,Angola,1750,0.0,9350528.0


In [16]:
output_folder = '../D3js-Visualization/data/'
output_file = output_folder + 'emissions_processed.csv'

df.to_csv(output_file, index=False)

In [17]:
# Download the Natural Earth dataset for countries with geometries
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Save the entire world as a GeoJSON file
output_folder = '../D3js-Visualization/data/'  # Replace with your desired folder path
output_file = output_folder + 'world_data.geojson'

world.to_file(output_file, driver='GeoJSON')


/var/folders/wp/wvs3l1cx7qj62cdcp32j1kpc0000gn/T/ipykernel_35714/2220538564.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
